In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather
import glob
import os
import pyarrow
from xclim import ensembles as ens
from xclim import subset
import matplotlib.pyplot as plt

In [2]:
folder = "/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/"

# Select variable

In [3]:
var = "tx_mean"

In [4]:
#Variable 
variable85="rcp85_"+var+"_seasonal"
variable45="rcp45_"+var+"_seasonal"
#variable="rcp[48]5_tg_mean_annual"

files85 = glob.glob(folder+"*"+variable85+".nc")
files45 = glob.glob(folder+"*"+variable45+".nc")
#ex: ACCESS1-3_rcp45_tn_mean_annual.nc
files85
files45

['/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/ACCESS1-3_rcp45_tx_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/BNU-ESM_rcp45_tx_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CanESM2_rcp45_tx_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/CMCC-CMS_rcp45_tx_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/GFDL-ESM2M_rcp45_tx_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/HadGEM2-CC_rcp45_tx_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/INM-CM4_rcp45_tx_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5A-LR_rcp45_tx_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/IPSL-CM5B-LR_rcp45_tx_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/MPI-ESM-LR_rcp45_tx_mean_seasonal.nc',
 '/scen3/scenario/netcdf/ouranos/portraits-clim-2.0/NorESM1-M_rcp45_tx_mean_seasonal.nc']

# FOR RCP 45

## Subsetting arrays by periods of time with Xclim

In [5]:
dsEns45= ens.create_ensemble(files45)
dsEns45.time.dt.season[0:4]
# Seasonal example 30 y means
listds = []
i=0
for s in dsEns45.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns45.sel(time=(dsEns45.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    #print(tmp1)
    i = i+1
    listds.append(tmp1)

0
1
2
3


## Loop to create all 4 dataframes

In [6]:
i = 0
df45list = []
for ds in listds:
    print ("Dataset number: ", i, "--------------")
    perc45 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df45 = perc45.to_dataframe()
    df452 = df45.reset_index()
    df453 = df452.loc[df452["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df453.groupby([df453.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp452 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df45C = dfp452.copy()
    df45C[var+"_p10"] = round(df45C[var+"_p10"] -273.15,2)
    df45C[var+"_p50"] = round(df45C[var+"_p50"] -273.15,2)
    df45C[var+"_p90"] = round(df45C[var+"_p90"] -273.15,2)
    print ("Transformed in Celsiuds and rounded")
    Region1i45 = df45C.drop(columns=["realization"])
    print ("pivoted")
    df45f = Region1i45.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    df45f.columns = [season + "_" + year + "rcp45_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df45list.append(df45f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
winter
Columns renamed
DF added to the list
Dataset number:  1 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
spring
Columns renamed
DF added to the list
Dataset number:  2 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
summer
Columns renamed
DF added to the list
Dataset number:  3 --------------
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
fall
Columns renamed
DF added to the list


In [7]:
#df45list


In [8]:
dfw45 = df45list[0].reset_index()
dfsp45 = df45list[1].reset_index()
dfsu45 = df45list[2].reset_index()
dff45 = df45list[3].reset_index()

mkey = ["lat","lon"]
dfse45 = dfw45.merge(dfsp45, on=mkey).merge(dfsu45, on=mkey).merge(dff45, on=mkey)
dfse45.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp45"+var+"_seasonal.feather")

In [9]:
dfse452 = dfse45.drop(columns=["winter_hist_rcp45_p10","winter_hist_rcp45_p50", "winter_hist_rcp45_p90", 
                               "spring_hist_rcp45_p10", "spring_hist_rcp45_p50", "spring_hist_rcp45_p90", 
                               "summer_hist_rcp45_p10","summer_hist_rcp45_p50", "summer_hist_rcp45_p90", 
                               "fall_hist_rcp45_p10","fall_hist_rcp45_p50", "fall_hist_rcp45_p90"])
dfse452


,lat,lon,winter_t2050_rcp45_p10,winter_t2080_rcp45_p10,winter_t2050_rcp45_p50,winter_t2080_rcp45_p50,winter_t2050_rcp45_p90,winter_t2080_rcp45_p90,spring_t2050_rcp45_p10,spring_t2080_rcp45_p10,...,summer_t2050_rcp45_p50,summer_t2080_rcp45_p50,summer_t2050_rcp45_p90,summer_t2080_rcp45_p90,fall_t2050_rcp45_p10,fall_t2080_rcp45_p10,fall_t2050_rcp45_p50,fall_t2080_rcp45_p50,fall_t2050_rcp45_p90,fall_t2080_rcp45_p90
0,40.041039,-89.045212,2.87,3.20,5.08,5.64,7.39,7.99,17.45,18.26,...,31.90,32.66,33.64,34.50,19.84,20.22,21.53,22.13,23.32,24.22
1,40.041039,-88.961884,2.86,3.19,5.06,5.62,7.36,7.97,17.43,18.24,...,31.90,32.67,33.63,34.50,19.83,20.21,21.52,22.12,23.31,24.20
2,40.041039,-88.878548,2.85,3.17,5.03,5.60,7.33,7.93,17.40,18.21,...,31.91,32.67,33.63,34.49,19.83,20.20,21.51,22.11,23.30,24.20
3,40.041039,-88.795219,2.77,3.09,4.93,5.51,7.23,7.83,17.29,18.11,...,31.86,32.62,33.58,34.43,19.75,20.12,21.44,22.03,23.23,24.12
4,40.041039,-88.711891,2.77,3.09,4.92,5.50,7.21,7.82,17.30,18.12,...,31.91,32.67,33.62,34.47,19.78,20.15,21.47,22.06,23.26,24.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-16.68,-15.56,-13.81,-12.85,-10.69,-9.96,-10.61,-10.24,...,7.11,6.92,9.45,8.88,-3.74,-3.44,-1.78,-1.35,-0.27,0.27
76202,66.623306,-61.546310,-16.87,-15.75,-14.01,-13.05,-10.90,-10.17,-11.01,-10.64,...,6.68,6.47,9.03,8.46,-4.18,-3.88,-2.22,-1.81,-0.73,-0.19
76203,66.623306,-61.462978,-16.69,-15.58,-13.83,-12.86,-10.73,-10.00,-10.72,-10.35,...,6.96,6.74,9.30,8.74,-3.90,-3.61,-1.94,-1.54,-0.46,0.08
76204,66.623306,-61.379650,-16.62,-15.50,-13.76,-12.78,-10.67,-9.94,-10.61,-10.24,...,7.05,6.82,9.38,8.84,-3.81,-3.52,-1.85,-1.46,-0.38,0.16


In [10]:
dfsew = dfse45[["winter_hist_rcp45_p50"]]
dfsesp = dfse45[["spring_hist_rcp45_p50"]]
dfsesu = dfse45[["summer_hist_rcp45_p50"]]
dfsefa = dfse45[["fall_hist_rcp45_p50"]]

# FOR RCP 85

## Subsetting arrays by periods of time with Xclim

In [11]:
dsEns85= ens.create_ensemble(files85)
dsEns85.time.dt.season[0:4]
# Seasonal example 30 y means
listds85 = []
i=0
for s in dsEns85.time.dt.season[0:4]:
    print (i)
    tmp1 = dsEns85.sel(time=(dsEns85.time.dt.year>=1981))
    tmp1 = tmp1.sel(time=(tmp1.time.dt.season==s))
    print(tmp1)
    i = i+1
    listds85.append(tmp1)

0
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416, realization: 11, time: 120)
Coordinates:
  * lat          (lat) float32 66.62331 66.53998 66.45665 ... 40.12437 40.04104
  * time         (time) datetime64[ns] 1981-12-01 1982-12-01 ... 2100-12-01
  * lon          (lon) float32 -89.04521 -88.96188 ... -54.54659 -54.46326
  * realization  (realization) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    tx_mean      (realization, time, lat, lon) float32 dask.array<chunksize=(1, 1, 320, 416), meta=np.ndarray>
Attributes:
    Conventions:     CF-1.5
    title:           ACCESS1-3 model output prepared for CMIP5 historical
    history:         CMIP5 compliant file produced from raw ACCESS model outp...
    institution:     CSIRO (Commonwealth Scientific and Industrial Research O...
    source:          ACCESS1-3 2011. Atmosphere: AGCM v1.0 (N96 grid-point, 1...
    redistribution:  Redistribution prohibited. For internal use only.
1
<xarray.Dataset>
Dimensions:      (lat: 320, lon: 416,

## Loop to create all 4 dataframes

In [12]:
i = 0
df85list = []
for ds in listds85:
    print ("Dataset number: ", i)
    perc85 = ens.ensemble_percentiles(ds)
    print ("Percentiles Obteined")
    df85 = perc85.to_dataframe()
    df852 = df85.reset_index()
    df853 = df852.loc[df852["realization"] == 0].dropna()
    print ("Converted to DF")
    year_groups = {y:0 for y in range(1980,2011)}
    year_groups.update({y:1 for y in range(2041,2071)})
    year_groups.update({y:2 for y in range(2071,2101)})
    dfp = df853.groupby([df853.time.dt.year.map(year_groups), "lat","lon", "realization"]).mean()
    dfp852 =  dfp.reset_index()
    print ("Split in 3 different time periods")
    df85C = dfp852.copy()
    df85C[var+"_p10"] = round(df85C[var+"_p10"] -273.15,2)
    df85C[var+"_p50"] = round(df85C[var+"_p50"] -273.15,2)
    df85C[var+"_p90"] = round(df85C[var+"_p90"] -273.15,2)
    print ("Transformed in Celsiuds and rounded")
    Region1i85 = df85C.drop(columns=["realization"])
    print ("pivoted")
    df85f = Region1i85.pivot_table(index=["lat","lon"], columns="time")
    print ("DF pivoted")
    if i== 0: season = "winter" 
    elif i == 1: season = "spring"
    elif i == 2: season = "summer"
    elif i == 3: season = "fall"
    print (season)
    df85f.columns = [season + "_" + year + "rcp85_p"+p
                     for p in ["10", "50", "90"]
                     for year in ["hist_","t2050_", "t2080_"]]
    print("Columns renamed")
    df85list.append(df85f)
    print ("DF added to the list")
    i = i + 1
    

Dataset number:  0
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
winter
Columns renamed
DF added to the list
Dataset number:  1
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
spring
Columns renamed
DF added to the list
Dataset number:  2
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
summer
Columns renamed
DF added to the list
Dataset number:  3
Percentiles Obteined
Converted to DF
Split in 3 different time periods
Transformed in Celsiuds and rounded
pivoted
DF pivoted
fall
Columns renamed
DF added to the list


In [13]:
df85list


[                      winter_hist_rcp85_p10  winter_t2050_rcp85_p10  \
 lat       lon                                                         
 40.041039 -89.045212                   0.61                    3.53   
           -88.961884                   0.61                    3.52   
           -88.878548                   0.60                    3.51   
           -88.795219                   0.52                    3.42   
           -88.711891                   0.53                    3.43   
 ...                                     ...                     ...   
 66.623306 -61.629639                 -20.98                  -15.10   
           -61.546310                 -21.17                  -15.28   
           -61.462978                 -20.99                  -15.09   
           -61.379650                 -20.91                  -15.01   
           -61.296322                 -20.45                  -14.55   
 
                       winter_t2080_rcp85_p10  winter_hist_rcp

In [14]:
dfw85 = df85list[0].reset_index()
dfsp85 = df85list[1].reset_index()
dfsu85 = df85list[2].reset_index()
dff85 = df85list[3].reset_index()
mkey = ["lat","lon"]
dfse85 = dfw85.merge(dfsp85, on=mkey).merge(dfsu85, on=mkey).merge(dff85, on=mkey)
dfse85

,lat,lon,winter_hist_rcp85_p10,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_hist_rcp85_p50,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_hist_rcp85_p90,winter_t2050_rcp85_p90,...,summer_t2080_rcp85_p90,fall_hist_rcp85_p10,fall_t2050_rcp85_p10,fall_t2080_rcp85_p10,fall_hist_rcp85_p50,fall_t2050_rcp85_p50,fall_t2080_rcp85_p50,fall_hist_rcp85_p90,fall_t2050_rcp85_p90,fall_t2080_rcp85_p90
0,40.041039,-89.045212,0.61,3.53,5.34,3.23,5.86,7.54,5.41,8.14,...,37.26,17.60,20.72,22.61,18.98,22.55,24.71,20.71,24.41,26.70
1,40.041039,-88.961884,0.61,3.52,5.33,3.22,5.84,7.52,5.39,8.12,...,37.26,17.60,20.71,22.60,18.98,22.55,24.70,20.70,24.40,26.69
2,40.041039,-88.878548,0.60,3.51,5.32,3.20,5.82,7.49,5.35,8.09,...,37.25,17.60,20.71,22.59,18.98,22.55,24.69,20.70,24.38,26.68
3,40.041039,-88.795219,0.52,3.42,5.23,3.11,5.74,7.40,5.25,7.99,...,37.19,17.52,20.63,22.52,18.90,22.47,24.61,20.62,24.31,26.60
4,40.041039,-88.711891,0.53,3.43,5.24,3.11,5.73,7.39,5.23,7.97,...,37.23,17.55,20.66,22.55,18.94,22.50,24.64,20.65,24.33,26.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-20.98,-15.10,-11.30,-18.24,-11.67,-7.89,-15.53,-8.71,...,10.87,-5.19,-2.90,-2.34,-3.54,-1.09,0.39,-1.87,0.57,2.52
76202,66.623306,-61.546310,-21.17,-15.28,-11.50,-18.43,-11.89,-8.12,-15.72,-8.93,...,10.44,-5.62,-3.35,-2.79,-3.96,-1.54,-0.07,-2.30,0.10,2.04
76203,66.623306,-61.462978,-20.99,-15.09,-11.32,-18.26,-11.71,-7.96,-15.54,-8.76,...,10.72,-5.33,-3.06,-2.51,-3.67,-1.27,0.19,-2.01,0.37,2.29
76204,66.623306,-61.379650,-20.91,-15.01,-11.25,-18.18,-11.65,-7.90,-15.47,-8.70,...,10.81,-5.23,-2.98,-2.43,-3.56,-1.19,0.26,-1.91,0.45,2.35


In [15]:
dfse85.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/rcp85"+var+"_seasonal.feather")

# Merge df45 and 85

# Extract hist

In [16]:
dfse852 = dfse85.drop(columns=["winter_hist_rcp85_p10","winter_hist_rcp85_p50", "winter_hist_rcp85_p90", 
                               "spring_hist_rcp85_p10", "spring_hist_rcp85_p50", "spring_hist_rcp85_p90", 
                               "summer_hist_rcp85_p10","summer_hist_rcp85_p50", "summer_hist_rcp85_p90", 
                               "fall_hist_rcp85_p10","fall_hist_rcp85_p50", "fall_hist_rcp85_p90"])
dfse852


,lat,lon,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_t2050_rcp85_p90,winter_t2080_rcp85_p90,spring_t2050_rcp85_p10,spring_t2080_rcp85_p10,...,summer_t2050_rcp85_p50,summer_t2080_rcp85_p50,summer_t2050_rcp85_p90,summer_t2080_rcp85_p90,fall_t2050_rcp85_p10,fall_t2080_rcp85_p10,fall_t2050_rcp85_p50,fall_t2080_rcp85_p50,fall_t2050_rcp85_p90,fall_t2080_rcp85_p90
0,40.041039,-89.045212,3.53,5.34,5.86,7.54,8.14,10.44,18.19,19.48,...,33.03,35.17,34.88,37.26,20.72,22.61,22.55,24.71,24.41,26.70
1,40.041039,-88.961884,3.52,5.33,5.84,7.52,8.12,10.42,18.18,19.46,...,33.03,35.17,34.88,37.26,20.71,22.60,22.55,24.70,24.40,26.69
2,40.041039,-88.878548,3.51,5.32,5.82,7.49,8.09,10.38,18.15,19.44,...,33.03,35.17,34.87,37.25,20.71,22.59,22.55,24.69,24.38,26.68
3,40.041039,-88.795219,3.42,5.23,5.74,7.40,7.99,10.28,18.06,19.34,...,32.98,35.12,34.81,37.19,20.63,22.52,22.47,24.61,24.31,26.60
4,40.041039,-88.711891,3.43,5.24,5.73,7.39,7.97,10.27,18.07,19.36,...,33.03,35.17,34.85,37.23,20.66,22.55,22.50,24.64,24.33,26.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-15.10,-11.30,-11.67,-7.89,-8.71,-3.36,-9.66,-7.95,...,7.50,8.15,9.84,10.87,-2.90,-2.34,-1.09,0.39,0.57,2.52
76202,66.623306,-61.546310,-15.28,-11.50,-11.89,-8.12,-8.93,-3.60,-10.07,-8.37,...,7.06,7.71,9.43,10.44,-3.35,-2.79,-1.54,-0.07,0.10,2.04
76203,66.623306,-61.462978,-15.09,-11.32,-11.71,-7.96,-8.76,-3.46,-9.78,-8.08,...,7.33,7.98,9.71,10.72,-3.06,-2.51,-1.27,0.19,0.37,2.29
76204,66.623306,-61.379650,-15.01,-11.25,-11.65,-7.90,-8.70,-3.43,-9.68,-7.99,...,7.42,8.07,9.81,10.81,-2.98,-2.43,-1.19,0.26,0.45,2.35


## Getting historic mean for each season

In [17]:
### Function
def mean_season(season, dfse45, dfse85):
    dfw45 = dfse45[["lat", "lon", season+"_hist_rcp45_p50"]]
    dfw85 = dfse85[["lat", "lon", season+"_hist_rcp85_p50"]]
    dfsew = pd.merge(dfw45, dfw85, on=["lat","lon"])
    dfsew[season+"_hist_p50"] = round((dfsew[season+"_hist_rcp45_p50"]+ dfsew[season+"_hist_rcp85_p50"])/2, 2)
    dftgwh2 = dfsew.reset_index()
    dftgwh3 = dftgwh2[["lat", "lon", season+"_hist_p50"]]
    return (dftgwh3)


### Winter historic mean

In [18]:
dftgwh = mean_season("winter", dfse45, dfse85)
dftgwh

,lat,lon,winter_hist_p50
0,40.041039,-89.045212,3.23
1,40.041039,-88.961884,3.22
2,40.041039,-88.878548,3.20
3,40.041039,-88.795219,3.11
4,40.041039,-88.711891,3.10
...,...,...,...
76201,66.623306,-61.629639,-18.22
76202,66.623306,-61.546310,-18.40
76203,66.623306,-61.462978,-18.24
76204,66.623306,-61.379650,-18.16


### Spring historic mean

In [19]:
dftsph = mean_season("spring", dfse45, dfse85)
dftsph

,lat,lon,spring_hist_p50
0,40.041039,-89.045212,17.78
1,40.041039,-88.961884,17.76
2,40.041039,-88.878548,17.73
3,40.041039,-88.795219,17.63
4,40.041039,-88.711891,17.63
...,...,...,...
76201,66.623306,-61.629639,-10.54
76202,66.623306,-61.546310,-10.94
76203,66.623306,-61.462978,-10.66
76204,66.623306,-61.379650,-10.54


### Summer historic mean

In [20]:
dftsuh = mean_season("summer", dfse45, dfse85)
dftsuh

,lat,lon,summer_hist_p50
0,40.041039,-89.045212,29.87
1,40.041039,-88.961884,29.87
2,40.041039,-88.878548,29.87
3,40.041039,-88.795219,29.82
4,40.041039,-88.711891,29.86
...,...,...,...
76201,66.623306,-61.629639,7.52
76202,66.623306,-61.546310,7.10
76203,66.623306,-61.462978,7.36
76204,66.623306,-61.379650,7.44


### Fall historic mean

In [21]:
dftfah = mean_season("fall", dfse45, dfse85)
dftfah

,lat,lon,fall_hist_p50
0,40.041039,-89.045212,19.01
1,40.041039,-88.961884,19.01
2,40.041039,-88.878548,19.00
3,40.041039,-88.795219,18.92
4,40.041039,-88.711891,18.96
...,...,...,...
76201,66.623306,-61.629639,-3.62
76202,66.623306,-61.546310,-4.04
76203,66.623306,-61.462978,-3.75
76204,66.623306,-61.379650,-3.64


# Merge means with the 45 and 85 DF

In [22]:
dftgseall = dfse852.merge(dfse452, on=mkey).merge(dftfah, on=mkey).merge(dftsuh, on=mkey).merge(dftsph, on=mkey).merge(dftgwh, on=mkey)

In [23]:
dftgseall

,lat,lon,winter_t2050_rcp85_p10,winter_t2080_rcp85_p10,winter_t2050_rcp85_p50,winter_t2080_rcp85_p50,winter_t2050_rcp85_p90,winter_t2080_rcp85_p90,spring_t2050_rcp85_p10,spring_t2080_rcp85_p10,...,fall_t2050_rcp45_p10,fall_t2080_rcp45_p10,fall_t2050_rcp45_p50,fall_t2080_rcp45_p50,fall_t2050_rcp45_p90,fall_t2080_rcp45_p90,fall_hist_p50,summer_hist_p50,spring_hist_p50,winter_hist_p50
0,40.041039,-89.045212,3.53,5.34,5.86,7.54,8.14,10.44,18.19,19.48,...,19.84,20.22,21.53,22.13,23.32,24.22,19.01,29.87,17.78,3.23
1,40.041039,-88.961884,3.52,5.33,5.84,7.52,8.12,10.42,18.18,19.46,...,19.83,20.21,21.52,22.12,23.31,24.20,19.01,29.87,17.76,3.22
2,40.041039,-88.878548,3.51,5.32,5.82,7.49,8.09,10.38,18.15,19.44,...,19.83,20.20,21.51,22.11,23.30,24.20,19.00,29.87,17.73,3.20
3,40.041039,-88.795219,3.42,5.23,5.74,7.40,7.99,10.28,18.06,19.34,...,19.75,20.12,21.44,22.03,23.23,24.12,18.92,29.82,17.63,3.11
4,40.041039,-88.711891,3.43,5.24,5.73,7.39,7.97,10.27,18.07,19.36,...,19.78,20.15,21.47,22.06,23.26,24.15,18.96,29.86,17.63,3.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76201,66.623306,-61.629639,-15.10,-11.30,-11.67,-7.89,-8.71,-3.36,-9.66,-7.95,...,-3.74,-3.44,-1.78,-1.35,-0.27,0.27,-3.62,7.52,-10.54,-18.22
76202,66.623306,-61.546310,-15.28,-11.50,-11.89,-8.12,-8.93,-3.60,-10.07,-8.37,...,-4.18,-3.88,-2.22,-1.81,-0.73,-0.19,-4.04,7.10,-10.94,-18.40
76203,66.623306,-61.462978,-15.09,-11.32,-11.71,-7.96,-8.76,-3.46,-9.78,-8.08,...,-3.90,-3.61,-1.94,-1.54,-0.46,0.08,-3.75,7.36,-10.66,-18.24
76204,66.623306,-61.379650,-15.01,-11.25,-11.65,-7.90,-8.70,-3.43,-9.68,-7.99,...,-3.81,-3.52,-1.85,-1.46,-0.38,0.16,-3.64,7.44,-10.54,-18.16


In [24]:
dftgseall.to_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_seasonal.feather")

In [25]:
#dftgseall = pd.read_feather("/home/mlopez/EXEC/Processed data to clip with regions/"+var+"_annual.feather")

# Merge complete DF with Polygons for each spatial scale

## Dictionary - regions: column name, short name

In [26]:
short_dict = {"DDE_STF_20K_REG_FOR_VUE_S": ("NM_REG_FOR", "RF"), 
            "DDE_STF_20K_UA_PER_VUE_S": ("PER_NO_UA", "UA"), 
              "DOM_BIO": ("NOM", "DB"), 
              "REG_ECO": ("NOM", "RE"), 
              "SDOM_BIO": ("NOM", "SDB"), 
              "Secteurs_Operations_Regionales": ("D_GENERAL", "SOR"), 
              "SREG_ECO": ("NOM", "SRE"),  
              "territoire_guide": ("TER_GUIDE", "TG") 
              }

for region, (name, short) in short_dict.items():
    print(region, name, short)

DDE_STF_20K_REG_FOR_VUE_S NM_REG_FOR RF
DDE_STF_20K_UA_PER_VUE_S PER_NO_UA UA
DOM_BIO NOM DB
REG_ECO NOM RE
SDOM_BIO NOM SDB
Secteurs_Operations_Regionales D_GENERAL SOR
SREG_ECO NOM SRE
territoire_guide TER_GUIDE TG


In [27]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

In [28]:
d = { 'BAS-SAINT-LAURENT':"Bas-Saint-Laurent", 
     'SAGUENAY -LAC-SAINT-JEAN': "Saguenay -Lac-Saint-Jean",
     'CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES':"Capitale-Nationale-Chaudiere-Appalaches",
     'MAURICIE-CENTRE-DU-QUÉBEC':'Mauricie-Centre-du-Quebec','OUTAOUAIS':'Outaouais', 
     'ABITIBI-TEMISCAMINGUE':'Abitibi-Temiscamingue', 'COTE-NORD':'Cote-Nord',
     'NORD-DU-QUEBEC':'Nord-Du-Quebec', 'GASPESIE-ILES-DE-LA-MADELEINE':'Gaspesie-Iles-De-La-Madeleine', 
     'LANAUDIERE':'Lanaudiere','LAURENTIDES':"Laurentides", 
     "ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL":"Estrie-Monteregie-Laval-Montreal", 
     "é": "e", "É": "E", "à": "a", "è": "e", "Î": "i", "È": "E", "ô" : "o", "Ç":"C", "ç":"c",
       } 

In [29]:
for region, (name, short) in short_dict.items():
    print(region)
    dfpolyshape = from_geofeather('/home/mlopez/EXEC/Grids-polygons-regions/Grid-'+region+'.feather')
    dftp = pd.merge(dftgseall, dfpolyshape, on=["lat","lon"])
    print ("Merged with polygons")
    listTG = []
    for tg in dftp[name].unique().tolist():
        df2 = dftp[dftp[name] == tg]
        print (tg)
        if tg != None:
            listTG.append(df2)
        #print (listTG)
    for df in listTG:
        geometry = df["geometry"]
        crs = {'init': "epsg:4326"}
        gdf = GeoDataFrame(df, crs=crs, geometry=geometry)
        print (gdf[name].iloc[0])
        #Substitute filename accents
        gdf.to_file(replace_all("/home/mlopez/EXEC/GeoJsonMFFP/"+gdf[name].iloc[0], d)+"_"+var+"_seasonal.json", driver="GeoJSON")
    geometry = dftp["geometry"]
    crs = {'init': "epsg:4326"}
    gdf = GeoDataFrame(dftp, crs=crs, geometry=geometry)
    gdf.to_file("/home/mlopez/EXEC/GeoJsonMFFP/"+short+"_"+var+"_seasonal.json", driver="GeoJSON")

  warnings.warn(



DDE_STF_20K_REG_FOR_VUE_S
Merged with polygons
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC
ESTRIE-MONTÉRÉGIE-LAVAL-MONTRÉAL
OUTAOUAIS
LAURENTIDES
MAURICIE-CENTRE-DU-QUÉBEC
LANAUDIERE
CAPITALE-NATIONALE-CHAUDIÈRE-APPALACHES
ABITIBI-TEMISCAMINGUE
BAS-SAINT-LAURENT
GASPESIE-ILES-DE-LA-MADELEINE
SAGUENAY -LAC-SAINT-JEAN
COTE-NORD
NORD-DU-QUEBEC


  warnings.warn(



DDE_STF_20K_UA_PER_VUE_S
Merged with polygons
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352
05151
07151
07251
06151
03571
06452
06271
07351
07152
08151
06471
04151
03451
08152
07352
03171
07451
04352
04251
01171
03153
08351
02371
03351
08251
04351
01272
08451
02571
11161
09751
11262
08462
08651
02651
08751
08551
02471
11263
08762
02751
02664
09351
08652
02666
08763
09471
08764
02665
08665
02663
08562
08664
08666
02662
08663
09551
02661
09352


  warnings.warn(



...

Massif des monts Groulx


ERROR:fiona._env:/home/mlopez/EXEC/GeoJsonMFFP/Coteaux des lacs Matonipi et Jonquet_tx_mean_seasonal.json: No such file or directory


Coteaux des lacs Matonipi et Jonquet


  warnings.warn(



territoire_guide
Merged with polygons
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
1a
2c
3d
2b
2a
3c
3ab
4bc
4de
5ef
4f
4a
5jk
5bcd
5a
4gh
5hi
5g
6cdefg
6hi
6ab
6j
6mn
6kl
6opqr
